In [1]:
import win32api as wapi
import time
import pyautogui
import cv2
import numpy as np
import win32gui, win32ui, win32con
from itertools import chain

In [11]:
def record_click():
    state_left = win32api.GetKeyState(0x01)  # Left button down = 0 or 1. Button up = -127 or -128 
    state_right = win32api.GetKeyState(0x02)  # Right button down = 0 or 1. Button up = -127 or -128 
    while True: 
        a = win32api.GetKeyState(0x01) 
        b = win32api.GetKeyState(0x02) 

        if a != state_left:  # Button state changed 
            state_left = a 
            if a < 0:
                return ["Left", pyautogui.postion()]
                break

        if b != state_right:  # Button state changed 
            state_right = b 
            if b < 0: 
                return ["Right", pyautogui.postion()]
                break

def grab_screen(region=None):

    hwin = win32gui.GetDesktopWindow()

    if region:
            left, top = region[0]
            x2,y2=region[1]
            width = x2 - left + 1
            height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)
    
    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.fromstring(signedIntsArray, dtype='uint8')
    img.shape = (height, width, 4)

    #cv2.imwrite("imageFULL.png", img) 
    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())

    return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

def get_region():
    state_left = win32api.GetKeyState(0x01)  # Left button down = 0 or 1. Button up = -127 or -128 
    while True: 
        a = win32api.GetKeyState(0x01) 
        region=[]
        if a != state_left:# Button state changed 
            state_left = a 
            if a < 0:
                region.append(pyautogui.postion())
            else:
                region.append(pyautogui.postion())
        return region
        break
        


In [ ]:
def listen_action():
    while True:
        if wapi.GetAsyncKeyState(ord('C')):
            action=record_click()
            break
        elif wapi.GetAsyncKeyState(ord('R')):
            region=get_region()
            region = list(chain.from_iterable(region))
            